In [1]:
#tensorflow 2.2.2
#others newest
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras import Input, Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K

import tensorflow as tf
from glob import glob
import pickle
import numpy as np
import random
import os
import tqdm.keras as tk
# from keras_tqdm import TQDMNotebookCallback
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

/home/l6wu/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/l6wu/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/l6wu/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/l6wu/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWa

In [2]:
train_path = "./new_train/new_train"
test_path = "./new_val_in/new_val_in"

In [3]:
data_list = glob(os.path.join(train_path, '*'))
random.shuffle(data_list)
print(int(len(data_list)*1))
data_list = data_list[:len(data_list)]
print(int(len(data_list)*1))

train_list = data_list[:int(len(data_list)*0.9)-47]
valid_list = data_list[int(len(data_list)*0.9)+95:]
print(len(train_list))
print(len(valid_list))

205942
205942
185300
20500


In [4]:
batch_size = 100
epochs = 10
latent_dim = 256
num_encoder_tokens = 2
num_decoder_tokens = 2

In [5]:
def train_generator_y(train_list, batch_size):
    index = 0
    # shape of return data (batch_size, 19, [position + velocity])
    # shape of return data (batch_size, 30, [position + velocity])
    while True:
        encoder_input_data = np.zeros((batch_size, 19, 2), dtype='float32')
        decoder_input_data = np.zeros((batch_size, 30, 2), dtype='float32')
        decoder_target_data = np.zeros((batch_size, 30, 2), dtype='float32')
        for i in range(batch_size):
            train_path = train_list[index]
            with open(train_path, 'rb') as f:
                data = pickle.load(f)                
                agent_id = data['agent_id']
                idx = np.where(data["track_id"] == data["agent_id"])[0][0]
                
                v_in_x = data["v_in"][idx,:,1]
                p_in_x = data["p_in"][idx,:,1]
                v_out_x = data["v_out"][idx,:,1]
                p_out_x = data["p_out"][idx,:,1]
                
                encoder_input_data[i,:,0] = p_in_x
                encoder_input_data[i,:,1] = v_in_x
                
                decoder_input_data[i,0,0] = p_in_x[-1]
                decoder_input_data[i,0,1] = v_in_x[-1]
                decoder_input_data[i,1:,0] = p_out_x[:-1]
                decoder_input_data[i,1:,1] = v_out_x[:-1]
                
                decoder_target_data[i,:,0] = p_out_x
                decoder_target_data[i,:,1] = v_out_x
               
            index += 1
        if index == len(train_list):
            index = 0

        yield [encoder_input_data, decoder_input_data], decoder_target_data

def valid_generator_y(valid_list, batch_size):
    index = 0
    while True:
        encoder_input_data = np.zeros((batch_size, 19, 2), dtype='float32')
        decoder_input_data = np.zeros((batch_size, 30, 2), dtype='float32')
        decoder_target_data = np.zeros((batch_size, 30, 2), dtype='float32')
        for i in range(batch_size):
            valid_path = valid_list[index]
            with open(valid_path, 'rb') as f:
                data = pickle.load(f)                
                agent_id = data['agent_id']
                idx = np.where(data["track_id"] == data["agent_id"])[0][0]
                
                v_in_y = data["v_in"][idx,:,1]
                p_in_y = data["p_in"][idx,:,1]
                v_out_y = data["v_out"][idx,:,1]
                p_out_y = data["p_out"][idx,:,1]
                
                encoder_input_data[i,:,0] = p_in_y
                encoder_input_data[i,:,1] = v_in_y
                
                decoder_input_data[i,0,0] = p_in_y[-1]
                decoder_input_data[i,0,1] = v_in_y[-1]
                decoder_input_data[i,1:,0] = p_out_y[:-1]
                decoder_input_data[i,1:,1] = v_out_y[:-1]
                
                decoder_target_data[i,:,0] = p_out_y
                decoder_target_data[i,:,1] = v_out_y
                
            index += 1
        if index == len(valid_list):
            index = 0

        yield [encoder_input_data, decoder_input_data], decoder_target_data



In [6]:
#encoder portion
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True, activation="relu")
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

#decoder portion
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, activation="relu")
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='linear')
decoder_outputs = decoder_dense(decoder_outputs)

# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [7]:
train_gen_y = train_generator_y(train_list,100)
valid_gen_y = valid_generator_y(valid_list,100)

In [8]:
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 


model.compile(optimizer='adam', loss=root_mean_squared_error)
checkpointer = ModelCheckpoint(filepath='./lstm_seqtoseq_y_weights.hdf5', verbose=2, save_best_only=True, mode='min')

In [ ]:
hist_y = model.fit(train_gen_y,
                        verbose=2,
                        epochs=10,
                        validation_data=valid_gen_y,
                        steps_per_epoch=(len(train_list)/100),
                        validation_steps=(len(valid_list)/100),
                        callbacks=[checkpointer])

Epoch 1/10
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 00001: val_loss improved from inf to 171.98742, saving model to ./lstm_seqtoseq_y_weights.hdf5
1853/1853 - 426s - loss: 338.5666 - val_loss: 171.9874
Epoch 2/10

Epoch 00002: val_loss improved from 171.98742 to 103.02857, saving model to ./lstm_seqtoseq_y_weights.hdf5
1853/1853 - 424s - loss: 146.1898 - val_loss: 103.0286
Epoch 3/10

Epoch 00003: val_loss improved from 103.02857 to 68.06461, saving model to ./lstm_seqtoseq_y_weights.hdf5
1853/1853 - 428s - loss: 84.6343 - val_loss: 68.0646
Epoch 4/10

Epoch 00004: val_loss improved from 68.06461 to 41.17521, saving model to ./lstm_seqtoseq_y_weights.hdf5
1853/1853 - 554s - loss: 54.8671 - val_loss: 41.1752
Epoch 5/10


In [ ]:
TrainERR=hist_y.history['loss']
ValidERR=hist_y.history['val_loss']
# print('@%f, Minimun error:%f, at iteration: %i' % (hist.history['val_loss'][epoch-1], np.min(np.asarray(ValidERR)),np.argmin(np.asarray(ValidERR))+1))
print('drawing the training process...')
plt.figure(2)
plt.plot(range(1,10+1),TrainERR,'b',label='TrainERR')
plt.plot(range(1,10+1),ValidERR,'r',label='ValidERR')
plt.xlim([1,epochs])
plt.legend()
plt.xlabel('epoch')
plt.ylabel('error')
plt.grid(True)
fig1 = plt.gcf()
fig1.savefig('lstm_p+v_x.png', dpi=100)
plt.show()

In [ ]:
#encoder model inference
encoder_model = Model(encoder_inputs, encoder_states)

#decoder model inference
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)


decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [ ]:
def decode_sequence(input_seq):
    #input seq is a scene 
    # Encode the input as state vectors.
#     print("input seq shape is ",input_seq.shape)
    input_shaped = np.empty((1,19,2))
    input_shaped[0,:,:] = input_seq
#     encoder_model.summary()
#     input_shaped = np.reshape(input_seq, (input_seq.shape[0], 1, input_seq.shape[1]))
#     print("inpiut shaped is ", input_shaped)
    states_value = encoder_model.predict(input_shaped)
    
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0,0,:] = input_seq[len(input_seq)-1] 

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    index = 0
    decoded_sentence = ''
    decoded_array = np.empty((30,2))
    while index < 30:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        
#         print("output tokens are ", output_tokens[0][0])
        
        # Sample a token
        decoded_array[index,:] = output_tokens[0][0]

        
        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, :] = output_tokens
        # Update states
        states_value = [h, c]
    
        index += 1
    
    return decoded_array

In [ ]:
columns = []
columns.append("ID")
for i in range(30):
    num = "v"+str(i+1)
    columns.append(num)
    
df = pd.DataFrame(columns=columns)

In [ ]:
#testing portion
test_list = glob(os.path.join(test_path, '*'))
print(len(test_list))

for x in tqdm(test_list):
    test_x = np.empty((30,2))
    with open(x, 'rb') as f:
        data = pickle.load(f)
        new_row = []
        scene_id = data['scene_idx']
        new_row.append(scene_id)
        agent_id = data['agent_id']
        idx = np.where(data["track_id"] == data["agent_id"])[0][0]   
        input_data = np.empty((19,2))
        input_data[:,0] = data['p_in'][idx,:,1]
        input_data[:,1] = data['v_in'][idx,:,1]
        
#         input_data = input_data[..., np.newaxis]
        test_x = decode_sequence(input_data)
        flat = test_x.flatten()
        for i in range(30):
            new_row.append(flat[i*2])
#         for elem in flat:
#             new_row.append(elem)
        df_length = len(df)
        df.loc[df_length] = new_row

In [ ]:
df['ID'] = df['ID'].map(round)

In [ ]:
df

In [ ]:
columns_test = []
columns_test.append("ID")
for i in range(38):
    num = "v"+str(i+1)
    columns_test.append(num)
    
df_test = pd.DataFrame(columns=columns_test)


for x in test_list:
    with open(x, 'rb') as f:
        data = pickle.load(f)
        new_row = []
        scene_id = data['scene_idx']
        new_row.append(scene_id)
        agent_id = data['agent_id']
        idx = np.where(data["track_id"] == data["agent_id"])[0][0]   
        input_data = data['p_in'][idx,:,:]
        for i in range(38):
            new_row.append(input_data.flatten()[i])
        df_length = len(df_test)
        df_test.loc[df_length] = new_row


In [ ]:
df_test

In [ ]:
df.to_csv(r'./lstm_y.csv', index = False, header=True)